#        Movie Recommender 

This recommender algorithm provides the similar movies of a selected movie of intreset 

It's a "content based algorithm" which basically depends on the data selected.

Installing the libraries required for algorithm:

In [873]:
import pandas as pd 
import numpy as np
import sklearn as sk 

Selected a database in .cvs format from koggle.com

In [874]:
movies_data= pd.read_csv('/Users/tejusmacbookair/Downloads/movies.csv')

#movies_data.head() # provides the data present in movies_data file 

In [875]:
movies_data.head(1) #provides the first row of the data in movies_data file

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,758323,The Pope's Exorcist,Horror-Mystery-Thriller,en,Father Gabriele Amorth Chief Exorcist of the V...,5953.227,Screen Gems-2.0 Entertainment-Jesus & Mary-Wor...,2023-04-05,18000000.0,65675816.0,103.0,Released,Inspired by the actual files of Father Gabriel...,7.433,545.0,Russell Crowe-Daniel Zovatto-Alex Essoe-Franco...,spain-rome italy-vatican-pope-pig-possession-c...,/9JBEPLTPSm0d1mbEcLxULjJq9Eh.jpg,/hiHGRbyTcbZoLsYYkO4QiCLYe34.jpg,713704-296271-502356-1076605-1084225-1008005-9...


In [876]:
movies_data_sliced = movies_data.iloc[:5000] #It gives frist 5000 rows values (from 0 to 4999) from the data
# I sliced the data to reduce dataload

In [877]:
#movies_data_sliced.head(1)['credits'].values
# to check format andtvvalues of specific column (here: credits)

In [878]:
#movies_data_sliced.shape
#To get the dimesions of the data structure (rows, col)

Below are the attributes that helps us to draw similarities between the moives
1. id
2. title
3. genres
4. overview
5. Credits
6. keywords

In [879]:
#movies_data['original_language'].value_counts()
# to get the value count of distinct values in an attribute 

In [880]:
movies_data_sliced = movies_data_sliced[['id','title','genres','overview','credits','keywords','poster_path']]
# conisdered only took reuired attributes 

In [881]:
movies_data_sliced.info() #to get detail count of each col values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           5000 non-null   int64 
 1   title        5000 non-null   object
 2   genres       4934 non-null   object
 3   overview     4939 non-null   object
 4   credits      4907 non-null   object
 5   keywords     4264 non-null   object
 6   poster_path  4984 non-null   object
dtypes: int64(1), object(6)
memory usage: 273.6+ KB


# Data-preprocessing

In [882]:
movies_data_sliced.isnull().sum() # To get total no.of null values in each col

id               0
title            0
genres          66
overview        61
credits         93
keywords       736
poster_path     16
dtype: int64

In [883]:
movies_data_sliced.dropna(inplace=True) # droping the null values

In [884]:
movies_data_sliced.duplicated().sum() #checking the duplicate values 

38

In [885]:
movies_data_sliced.drop_duplicates(inplace=True)#drop duplicates 

In [886]:
#movies_data_sliced.iloc[0].genres #checking the data format of geners

In [887]:
# string_to_list is the function converts string with (-) to list 
def string_to_list (obj):
    li = []
    li = obj.split('-')
    return li


In [888]:
#applying the string_to_list fucntion to genres,keywords,credits
movies_data_sliced['genres'] =movies_data_sliced['genres'].apply(string_to_list)
movies_data_sliced['keywords']=movies_data_sliced['keywords'].apply(string_to_list)
movies_data_sliced['credits']=movies_data_sliced['credits'].apply(string_to_list)

In [889]:
movies_data_sliced.iloc[0].keywords

['spain',
 'rome italy',
 'vatican',
 'pope',
 'pig',
 'possession',
 'conspiracy',
 'devil',
 'exorcist',
 'skepticism',
 'catholic priest',
 '1980s',
 'supernatural horror']

In [890]:
#movies_data_sliced.iloc[0].credits

In [891]:
#movies_data_sliced.iloc[0].overview

In [892]:
movies_data_sliced['overview']

0       Father Gabriele Amorth Chief Exorcist of the V...
1       Barbie and Ken are having the time of their li...
2       When a new threat capable of destroying the en...
3       Super-Hero partners Scott Lang and Hope van Dy...
4       After dominating the boxing world Adonis Creed...
                              ...                        
4994    Rosario Tijeras has been abused by men all her...
4995    Along with his new friends a teenager who was ...
4996    The modern-day story focuses on two beautiful ...
4997    After the Cold War a breakaway Russian republi...
4998    Dr. Malcolm Sayer a shy research physician use...
Name: overview, Length: 4180, dtype: object

In [893]:
#below function is used to convert sentence to list where (' ')
def sentence_to_list (obj):
    li = obj.split()
    return li

In [894]:
#applied the sentence_to_list function 
movies_data_sliced['overview']=movies_data_sliced['overview'].apply(sentence_to_list)

In [895]:
#below function removes space in each element of the list
def replace_spaces_in_listelemnts(obj):
    li = [element.replace(' ', '') for element in obj]
    return li

#tejasri kandimalla

In [896]:
#replace_space_in_listelemnts is applied all list types
movies_data_sliced['genres'] =movies_data_sliced['genres'].apply(replace_spaces_in_listelemnts)
movies_data_sliced['keywords'] =movies_data_sliced['keywords'].apply(replace_spaces_in_listelemnts)
movies_data_sliced['credits']=movies_data_sliced['credits'].apply(replace_spaces_in_listelemnts)
movies_data_sliced['overview']=movies_data_sliced['overview'].apply(replace_spaces_in_listelemnts)

In [897]:
movies_data_sliced.head(3)

,id,title,genres,overview,credits,keywords,poster_path
0,758323,The Pope's Exorcist,"[Horror, Mystery, Thriller]","[Father, Gabriele, Amorth, Chief, Exorcist, of...","[RussellCrowe, DanielZovatto, AlexEssoe, Franc...","[spain, romeitaly, vatican, pope, pig, possess...",/9JBEPLTPSm0d1mbEcLxULjJq9Eh.jpg
1,346698,Barbie,"[Comedy, Adventure, Fantasy]","[Barbie, and, Ken, are, having, the, time, of,...","[MargotRobbie, RyanGosling, AmericaFerrera, Ar...","[feminism, patriarchy, satire, basedontoy, fem...",/iuFNMS8U5cb6xfzi51Dbkovj7vM.jpg
2,667538,Transformers: Rise of the Beasts,"[Action, Adventure, ScienceFiction]","[When, a, new, threat, capable, of, destroying...","[AnthonyRamos, DominiqueFishback, LunaLaurenVe...","[peru, alien, endoftheworld, basedoncartoon, b...",/gPbM0MK8CP8A174rmUwGsADNYKD.jpg


In [898]:
#Combaining all the lists type attributes to concat_tags
movies_data_sliced['concat_tags'] = movies_data_sliced['genres'] + movies_data_sliced['keywords'] + movies_data_sliced['credits'] + movies_data_sliced['overview']

In [899]:
#movies_data_sliced.head(1)

In [900]:
#spliting the data for fetching the paster path
movies_poster_data = movies_data_sliced[['id','poster_path']]

In [901]:
#copying the data required into new_movie_data file 
new_movie_data = movies_data_sliced[['id', 'title', 'concat_tags']]

In [902]:
new_movie_data.head(1)

,id,title,concat_tags
0,758323,The Pope's Exorcist,"[Horror, Mystery, Thriller, spain, romeitaly, ..."


In [903]:
#below function is used to convert list to a string 
def list_to_string_sentence(obj):
    return ' '.join(obj)    

In [904]:
# alternative for above funtion
#new_movie_data['concat_tags']=new_movie_data['concat_tags'].apply(lambda x:" ".join(x))

In [905]:
new_movie_data['concat_tags'] = new_movie_data['concat_tags'].apply(list_to_string_sentence)

/var/folders/11/5zlk47f16_q_7km36xfpv36r0000gn/T/ipykernel_2741/951056360.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movie_data['concat_tags'] = new_movie_data['concat_tags'].apply(list_to_string_sentence)


In [906]:
#new_movie_data['concat_tags'][0]

In [907]:
# convert everything to lower case 
def convert_to_lowercase(obj):
    return obj.lower()

In [908]:
new_movie_data['concat_tags'] = new_movie_data['concat_tags'].apply(convert_to_lowercase)

/var/folders/11/5zlk47f16_q_7km36xfpv36r0000gn/T/ipykernel_2741/2681815881.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movie_data['concat_tags'] = new_movie_data['concat_tags'].apply(convert_to_lowercase)


In [909]:
new_movie_data['concat_tags'][0]

"horror mystery thriller spain romeitaly vatican pope pig possession conspiracy devil exorcist skepticism catholicpriest 1980s supernaturalhorror russellcrowe danielzovatto alexessoe franconero peterdesouza feighoney laurelmarsden cornelljohn ryano'grady biancabardoe santibayón palomabloyd alessandrogruttadauria riverhawkins jordicollet carriemunro marcvelasco edwardharper jones matthewsim victorsolé tombonington andreadugoni edwhite lailabarwick gennarodiana pabloraybould ralphineson derekcarroll ellacannon father gabriele amorth chief exorcist of the vatican investigates a young boy's terrifying possession and ends up uncovering a centuries-old conspiracy the vatican has desperately tried to keep hidden."

# Recommender Algotihm 

Import necessary libraries to build the logic based on the preprocessed data


In [910]:
import nltk
from nltk.stem.porter import PorterStemmer
p_s = PorterStemmer()

The below function is to perform stemming on a given input string (obj). Stemming is a natural language processing technique which is used to reduce words to their base or root form. 

In [911]:
def stem(obj):
    li =[]
    for i in obj.split():
        li.append(p_s.stem(i))
    return " ".join(li)


In [912]:
new_movie_data['concat_tags'] = new_movie_data['concat_tags'].apply(stem)

/var/folders/11/5zlk47f16_q_7km36xfpv36r0000gn/T/ipykernel_2741/595285186.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movie_data['concat_tags'] = new_movie_data['concat_tags'].apply(stem)


In [913]:
#len(new_movie_data) #provides the number od rows

CountVectorizer is a text preprocessing and feature extraction technique provided by scikit-learn, a popular machine learning library in Python. It is commonly used to convert a collection of text documents into a matrix of token (word) counts, which can then be used as input for various machine learning algorithms.

In [914]:
#from sklearn.feature_extraction.text import CountVectorizer
#c_v = CountVectorizer(max_features = 5000, stop_words='english')
# here max_fetures : most frequently used 5000 words limit
# stop words : to remove the is an the etc words 

In [915]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v = TfidfVectorizer(max_features=15000, stop_words='english')

In [916]:
#vectors = c_v.fit_transform(new_movie_data['concat_tags']).toarray()
# tranforming the contact_tags senteces wordsvinto an array named vector

In [917]:
vectors = tfidf_v.fit_transform(new_movie_data['concat_tags']).toarray()

In [918]:
#c_v.fit_transform(new_movie_data['concat_tags']).toarray().shape
#gives the shape of the vector

In [919]:
tfidf_v.fit_transform(new_movie_data['concat_tags']).toarray().shape

(4180, 15000)

In [920]:
vectors #print the vector structure 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [921]:
for element in tfidf_v.get_feature_names_out():
    print (element)

007
10
100
1000
100000
10thcenturi
11
11th
12
12thcenturi
13
13th
14
14thcenturi
15
15thcenturi
16
16th
16thcenturi
17
17th
17thcenturi
18
1800
18th
18thcenturi
19
1910
1920
1920s
1930
1940
1942
1945
1950
1955
1960
1962
1963
1969
1970
1972
1973
1976
1977
1979
1980
1980s
1985
1986
1987
1988
1990
1991
1992
1994
1996
1997
1999
19th
19thcenturi
1stcenturi
20
200
2000
2001
2009
2010
2016
2019
2020
2029
2030
2040
2050
2060
2070
20th
21
22
22nd
24
25
26
27
30
300
39
3d
40
47
50
500
50cent
70
911
aaron
aaronbeeln
aaroncraven
aarondougla
aaroneckhart
aaronfor
aaronhendri
aaronj
aaronjackson
aaronlaplant
aaronlazar
aaronlustig
aaronmonaghan
aaronneil
aaronpaul
aaronpearl
aaronsaxton
aarontaylor
aarontoney
aaronv
aaronyoo
aasifmandvi
abandon
abandonedhous
abb
abbeyle
abbi
abbiecornish
abbijacobson
abbycraden
abbyelliott
abbyquinn
abbyryderfortson
abdoulayengom
abduct
abdulayoola
abdulsalaamelrazzac
abe
abeckas
abel
abevigoda
abhisinha
abiadeyemi
abigail
abigailbreslin
abil
abl
abo
aboard
abort
ab

In [922]:
#for element in c_v.get_feature_names_out():
    #print (element)
#provides the 5000 elements of the cv features 


"cosine_similarity" function is used to compute the cosine similarity between two or more vectors.

"The normalize" function is used to normalize (scale) input data to have unit norm (length). When applied to a data matrix, it will normalize each row, making the rows have a length of 1. 

Normalise is used as the cosine similarity is based on the direction of the vectors.

In [923]:
from sklearn.metrics.pairwise import cosine_similarity
#from sklearn.preprocessing import normalize

In [924]:
new_movie_data.isnull().any()# checks any null data present

id             False
title          False
concat_tags    False
dtype: bool

In [925]:
has_nan = np.isnan(vectors).any() #checks for any null value in vector 
has_nan

False

In [926]:
has_inf = np.isinf(vectors).any() # checks for any infinte values in vector
has_inf

False

In [927]:
similarity_distance = cosine_similarity(vectors) # the distane between the each movie to 5000 other movies

/opt/homebrew/Caskroom/miniforge/base/envs/env_perceptron/lib/python3.11/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [928]:
similarity_distance.shape

(4180, 4180)

In [929]:
[sum(v) for v in vectors if sum(v)==0.0] 

[]

In [930]:
sorted(list(enumerate(similarity_distance[0])),reverse=True, key=lambda x:x[1]) [1:6]

# sorted desecding order and first five movies

[(3804, 0.22107063033917013),
 (1206, 0.18076532602593026),
 (2971, 0.15239987805349736),
 (1657, 0.14976582741491057),
 (148, 0.1422010884042112)]

In [931]:
def recommend(movie):
    movie_index = new_movie_data[new_movie_data['title'] == movie].index[0]
    distance = similarity_distance[movie_index]
    movie_list = sorted(list(enumerate(distance)),reverse=True, key=lambda x:x[1]) [1:6]
    
    for i in movie_list:
        print(new_movie_data.iloc[i[0]].title)

In [932]:
recommend("Barbie")

Barbie: A Fairy Secret
Barbie: A Fashion Fairytale
Barbie & Her Sisters in A Pony Tale
Barbie: Princess Adventure
The Barbie Diaries


In [933]:
import pickle
#pickle.dump(new_movie_data.open('movietitles.pkl','wb') )
# Assuming `new_movie_data` is your Pandas DataFrame containing movie data
with open('movietitles.pkl', 'wb') as file:
    pickle.dump(new_movie_data, file)


In [934]:
#new_movie_data.title

In [935]:
with open('movie_dictionary_list.pkl', 'wb') as file:
    pickle.dump(new_movie_data.to_dict(orient='records'), file)

In [936]:
with open('simliar_movie_list.pkl', 'wb') as file:
    pickle.dump(similarity_distance, file)

In [937]:
#new_movie_data.to_dict(orient='records') 

In [938]:
with open('movies_poster_data.pkl', 'wb') as file:
    pickle.dump(movies_poster_data, file)